**Importation of needed Libraries**

In [1]:
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Automatically created module for IPython interactive environment


**Dataset Importaion and Processing**

The dataset - lfw_people were imported from sklearn.datasets. The label to predict is the identity of the individuals in the image (image recognition).

In [2]:
# Downloading and loading the dataset as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)


# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


**Splitting data into a training set and a test set with test_size of 25%, and printing the resulting input variables X**

In [3]:
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_test.shape)
print(X_train.shape)

(322, 1850)
(966, 1850)


**Setting parameters for the learner (Deep Convolutional Neural Network - CNN).**

In [4]:
#Import needed libraries
from tensorflow import keras
from tensorflow.keras import layers

X_train = X_train.reshape(len(X_train), 50, 37, 1)        #50 * 37 to add up to 1850
X_test = X_test.reshape(len(X_test), 50, 37, 1)

print("Fitting the CNN to the training set")

t0 = time()
 
output_classes = 7

model=keras.Sequential([
    keras.Input(shape=(50,37,1)),
    layers.Conv2D(32, kernel_size=(3,3), activation='relu'),   #The Conv2D has a kernel_size of (3,3) that is weight and height of 3
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),   #The activation function is relu, therefore 0 will be returned if any negative input is received
    layers.MaxPooling2D(pool_size = (2,2)),      #Has a pool size of (2,2) to further reduce the spatial information by half.
    layers.Flatten(),
    layers.Dropout(0.5),                         #The dropout is set at 0.5, that is to regularize the data and dropout 50% during training to avoid overfitting the training data.
    layers.Dense(256, activation = 'relu'),
    layers.Dense(output_classes, activation="softmax"),  
  ])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


#Training the model
model.fit(X_train, y_train, epochs=100, batch_size=128)

print("\n", "done in %0.3fs" % (time() - t0))

Fitting the CNN to the training set


2022-11-09 01:42:45.460437: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-11-09 01:42:45.674984: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
/opt/conda/lib/python3.7/site-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


Epoch 1/100
8/8 [==============================] - 2s 95ms/step - loss: 19.8415 - accuracy: 0.2329
Epoch 2/100
8/8 [==============================] - 1s 83ms/step - loss: 2.0241 - accuracy: 0.2950
Epoch 3/100
8/8 [==============================] - 1s 83ms/step - loss: 1.5691 - accuracy: 0.4420
Epoch 4/100
8/8 [==============================] - 1s 84ms/step - loss: 1.4136 - accuracy: 0.5104
Epoch 5/100
8/8 [==============================] - 1s 86ms/step - loss: 1.2094 - accuracy: 0.5694
Epoch 6/100
8/8 [==============================] - 1s 85ms/step - loss: 1.0157 - accuracy: 0.6542
Epoch 7/100
8/8 [==============================] - 1s 83ms/step - loss: 0.8368 - accuracy: 0.7226
Epoch 8/100
8/8 [==============================] - 1s 84ms/step - loss: 0.7952 - accuracy: 0.7246
Epoch 9/100
8/8 [==============================] - 1s 84ms/step - loss: 0.6827 - accuracy: 0.7640
Epoch 10/100
8/8 [==============================] - 1s 84ms/step - loss: 0.5716 - accuracy: 0.7981
Epoch 11/100
8/8 [

**Evaluating the trained model**

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\n Test accuracy:', test_acc)

11/11 - 0s - loss: 0.5874 - accuracy: 0.8851

 Test accuracy: 0.8850931525230408


> **Result: With the chosen parameters of 100 Epochs and Batch_size of 128 in the deep CNN, we have an accuracy of 88%.**